In [1]:
from scipy.optimize import minimize
import numpy as np

## Projection of embedding into intersection of two hyper-spheres

Let $C_u$ and $R_u$ and $C_v$ and $R_v$ be the center and radius of two $K$-dimensional hyper-spheres. $C_u, C_v \in {\rm I\!R}^K$ and $R_u, R_k \in {\rm I\!R}$. Let $X_{uv}$ be the embedding of the edge $e=(u,v) \in E(G)$ and $\bar{X}_{uv}$ denote the projection of $X_{uv}$ in the overlapping region of the two hyper-spheres mentioned earlier. The optimization problem we are trying to solve here is as follows:
<br>

Minimize $ \left\Vert \bar{X}_{uv} - X_{uv} \right\Vert^2$ <br>
subject to $\left\Vert \bar{X}_{uv} - C_{u} \right\Vert^2 \leq R_u^2$  and  $\left\Vert \bar{X}_{uv} - C_{v} \right\Vert^2 \leq R_v^2$

In this problem, the optimization variable is $\bar{X}_{uv}$.
            



In [6]:
def _constraint(params, args):
    x = params
    c = args[1]
    r = args[2]
    val = r**2 - np.linalg.norm(np.array(x) - np.array(c))**2
    return val

def _func_objective(params, args):
    x = args[0]
    c_u = args[1]
    c_v = args[2]
    r_u = args[3]
    r_v = args[4]
    proj_distance = np.linalg.norm(params - x)
    return proj_distance**2

def _minimize_projection_loss(init_params, args, args_1, args_2):
    constraints = [{'type':'ineq', 'fun': _constraint, 'args':[args_1]},
                  {'type':'ineq', 'fun': _constraint, 'args':[args_2]}]
    params = minimize(_func_objective, [init_params],
                      args=args,
                      constraints=constraints)
    print(params)
    return params['x']

def project(embeddings, centers, radii, edge_map, nodes):
    edge_count = embeddings.shape[0]
    embed_dim = embeddings.shape[1]
    node_count = centers.shape[0]
    assert centers.shape[1] == embed_dim
    assert node_count == radii.shape[0]
    projected_embeddings = np.empty((edge_count, embed_dim))
    for i in range(edge_count):
        embed = embeddings[i]
        edge = edge_map[i]
        node_u = edge[0]
        node_v = edge[1]
        node_u_ind = np.where(nodes == node_u)[0][0]
        node_v_ind = np.where(nodes == node_v)[0][0]
        c_u = centers[node_u_ind]
        r_u = radii[node_u_ind]
        c_v = centers[node_v_ind]
        r_v = radii[node_v_ind]
        init_embed = np.random.rand(embed_dim)
        
        args = [embed, c_u, c_v, r_u, r_v]
        args_1 = [embed, c_u, r_u]
        args_2 = [embed, c_v, r_v]
        proj_embed = _minimize_projection_loss(init_embed, args, args_1, args_2)
        projected_embeddings[i] = proj_embed
    
    print(projected_embeddings.shape)
    return projected_embeddings

### Test for the above optimization problem

In [7]:
embeddings = np.array([[0.5, 0]])
print(embeddings.shape)
centers = np.array([[0, 0], [4, 0]])
print(centers.shape)
radii = np.array([[3], [3]])
print(radii.shape)
edge_map = {0:(0, 1)}
nodes = np.array([0, 1])
project(embeddings, centers, radii, edge_map, nodes)

(1, 2)
(2, 2)
(2, 1)
     fun: 0.24999991277866324
     jac: array([  9.99999825e-01,  -1.79734081e-04])
 message: 'Optimization terminated successfully.'
    nfev: 26
     nit: 6
    njev: 6
  status: 0
 success: True
       x: array([  9.99999905e-01,  -8.98746864e-05])
(1, 2)


array([[  9.99999905e-01,  -8.98746864e-05]])

In [24]:
x = [3, 0]
c_u = [0, 0]
c_v = [4, 0]
r_u = 3
r_v = 3
args = [x, c_u, c_v, r_u, r_v]
args_1 = [x, c_u, r_u]
args_2 = [x, c_v, r_v]
init_params = [2, 0]
x_proj = _minimize_projection_loss(init_params, args, args_1, args_2)
print(x_proj)

[{'type': 'ineq', 'fun': <function _constraint at 0x10648ed08>, 'args': [[[3, 0], [0, 0], 3]]}, {'type': 'ineq', 'fun': <function _constraint at 0x10648ed08>, 'args': [[[3, 0], [4, 0], 3]]}]
[[3, 0], [0, 0], [4, 0], 3, 3]
[3, 0]
[2. 0.]
[3, 0]
[2. 0.]
[3, 0]
[2.00000001 0.        ]
[3, 0]
[2.00000000e+00 1.49011612e-08]
[3, 0]
[3.25 0.  ]
[3, 0]
[3.25 0.  ]
[3, 0]
[3.25000001 0.        ]
[3, 0]
[3.25000000e+00 1.49011612e-08]
[3, 0]
[ 3.00961538e+00 -1.20355533e-08]
[3, 0]
[ 3.00961538e+00 -1.20355533e-08]
[3, 0]
[ 3.00961540e+00 -1.20355533e-08]
[3, 0]
[3.00961538e+00 2.86560792e-09]
[3, 0]
[ 3.00001536e+00 -2.38444555e-09]
[3, 0]
[ 3.00001536e+00 -2.38444555e-09]
[3, 0]
[ 3.00001537e+00 -2.38444555e-09]
[3, 0]
[3.00001536e+00 1.25167156e-08]
[3, 0]
[ 2.99999999e+00 -1.25321652e-08]
     fun: 2.0597520097055894e-16
     jac: array([3.07348336e-05, 1.01322701e-08])
 message: 'Optimization terminated successfully.'
    nfev: 17
     nit: 4
    njev: 4
  status: 0
 success: True
       x